<a href="https://colab.research.google.com/github/sidraAamir/CA_Project/blob/main/DL_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [12]:
import json
import torch
from transformers import RobertaTokenizer, RobertaModel
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu


In [13]:
# Load your JSON data
with open('/content/qa_data.json', 'r') as f:
    data = json.load(f)

# Convert data into lists
questions = [item['question'] for item in data]
answers = [item['answer'] for item in data]


In [14]:
# Load RoBERTa tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# Function to encode text
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)  # Get mean pooled embeddings
    return embeddings

# Generate embeddings for all questions and answers
question_embeddings = torch.cat([get_embeddings(q) for q in questions], dim=0)
answer_embeddings = torch.cat([get_embeddings(a) for a in answers], dim=0)


In [16]:
def find_best_answer(query):
    query_embedding = get_embeddings(query)
    # Compute cosine similarity between query and all question embeddings
    similarities = cosine_similarity(query_embedding, question_embeddings)[0]
    best_match_idx = similarities.argmax()  # Index of the best matching question
    return answers[best_match_idx]


In [31]:
# Example query
query = "what does registrar office do?"
response = find_best_answer(query)
print("Query:", query)
print("Response:", response)


Query: what does registrar office do?
Response: Plagiarism is using someone else’s work or ideas without proper acknowledgment. This includes copying text, paraphrasing without citation, and submitting someone else’s work as your own.


In [32]:
#sample_queries = questions[:10]  # Replace with any sample questions for evaluation
#sample_queries:['Which department manages course registration and schedules at Habib University?',
# 'What steps should I take if I encounter a problem with my class schedule?',
# 'Can you tell me where the Registrar’s Office is located at Habib University?',
# 'What can I do to fix issues with my course schedule if I’m unable to go to the Registrar’s Office?',
# 'How can I get in touch with the Registrar’s Office for general questions?',
# 'How do I arrange a time to meet with my professors at Habib University?',
# 'What are my options if my professor’s office hours don’t align with my schedule?',
# 'If I’m unable to come to campus, can I arrange a virtual meeting with my professor?',
# 'What kinds of topics can I bring up with my professor during office hours?',
# 'Who can I contact for academic assistance at Habib University?']
sample_queries:['At Habib University, who is in charge of managing course enrollments and timetables?',
 'If I face an issue with my schedule, what steps should I consider?',
 'Could you let me know the location of the Registrar’s Office at Habib University?',
 'Is there any way to address scheduling issues without going to the Registrar’s Office in person?',
 'What’s the best way to reach out to the Registrar’s Office if I have general questions?',
 'What’s the process for booking a meeting with professors at Habib University?',
 'What should I do if my availability doesn’t line up with my professor’s office hours?',
 'Are there options for meeting my professor virtually if I can’t make it to campus?',
 'What kind of concerns are appropriate to discuss with my professor during office hours?',
 'If I need help academically, who at Habib University should I get in touch with?','Which department manages course registration and schedules at Habib University?',
 'What steps should I take if I encounter a problem with my class schedule?',
 'Can you tell me where the Registrar’s Office is located at Habib University?',
 'What can I do to fix issues with my course schedule if I’m unable to go to the Registrar’s Office?',
 'How can I get in touch with the Registrar’s Office for general questions?',
 'How do I arrange a time to meet with my professors at Habib University?',
 'What are my options if my professor’s office hours don’t align with my schedule?',
 'If I’m unable to come to campus, can I arrange a virtual meeting with my professor?',
 'What kinds of topics can I bring up with my professor during office hours?',
 'Who can I contact for academic assistance at Habib University?']


print("Sample Queries:")
print(sample_queries)
predicted_answers = [find_best_answer(q) for q in sample_queries]
print("Predicted Answers:")
print(predicted_answers)
# Calculate BLEU scores
bleu_scores = [sentence_bleu([ref], pred) for ref, pred in zip(answers[:10], predicted_answers)]
avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
print("Average BLEU Score:", avg_bleu_score)


Sample Queries:
['Which office handles class schedules and course enrollment at Habib University?', 'What should I do if I have a problem with my class schedule?', "Where is the Registrar's Office located at Habib University?", 'How can I resolve issues with my course schedule if I can’t visit the Registrar’s Office?', 'How do I contact the Registrar’s Office for general inquiries?', 'How can I book office hours with my professors at Habib University?', 'What should I do if the professor’s office hours don’t match my availability?', 'Can I meet with my professor virtually if I can’t come to campus?', 'What types of issues can I discuss with my professor during office hours?', 'Who can I reach out to for academic help at Habib University?']
Predicted Answers:
["The Office of Academic Systems and Registrar (RO) is responsible for handling class schedules and course enrollment at Habib University. If you encounter any issues with scheduling or enrollment, you can visit the Registrar's Off